In [10]:
from pathlib import Path

In [11]:
database_file = Path("database.txt")

In [12]:
from typing import List, Dict

def read_database(keys: List[str]):
    to_return = {}
    if not database_file.exists():
        raise FileNotFoundError("Database file not found, please write some data first")
    
    with open(database_file, "rb") as f:
        lines = f.readlines()
        pairs = [line.decode("utf-8").split("=", 1) for line in lines]
        pairs.reverse() # reverse the list to return the latest value
        for key in keys:
            for k, v in pairs:
                if k == key:
                    to_return[key] = v.strip()
                    break
    return to_return

def write_database(data: Dict[str, str]):
    with open(database_file, "ab") as f:
        for key, value in data.items():
            f.write(f"{key}={value}\n".encode("utf-8"))

In [13]:
def clear_all():
    database_file.unlink(missing_ok=True)

In [18]:
clear_all()
write_database({"key1": "value1", "key2": "value2", "key3": "value3"})
read_database(["key1", "key2", "key3"])

In [ ]:
import timeit
import statistics as stats

def get_write_stmt(count: int):
    return f"write_database({{f'key{{i}}': f'value{{i}}' for i in range({count})}})"

def get_read_stmt(count: int, limit: int):
    return f"read_database([f'key{{i}}' for i in random.sample(range({limit}), {count})])"

for count in [10_000, 100_000, 1_000_000]:
    times = timeit.repeat(get_write_stmt(count), setup='clear_all()', globals=globals(), number=1, repeat=3)
    print(f'Time to write {count} records: {stats.mean(times)}s')

for limit in [10_000, 100_000, 1_000_000]:
    times = timeit.repeat(get_read_stmt(1000, limit=limit), setup=f'import random;clear_all();{get_write_stmt(count=limit)}', globals=globals(), number=1, repeat=3)
    print(f'Time to read 1000 random records from {limit} records: {stats.mean(times)}s')